编译器和解释器

In [1]:
# 命令式程序
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g 

print(fancy_func(1, 2, 3, 4))

10


符号式编程<br>
python作为解释型语言，按顺序执行代码，但效率可能不高<br>
符号式编程，是代码通常只在完全定义了过程之后才执行计算，可以利用编译器对不再需要的变量释放内存，或者将代码转化为一个完全等价的代码片段

In [2]:
def add_():
    return """
def add(a, b):
    return a + b
"""

def fancy_func_():
    return """
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g 
"""

def evoke_():
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
y = compile(prog, '', 'exec')  # 将字符串prog编译为Python字节码对象, 'exec'表示执行多行代码
exec(y)                        # 执行编译后的字节码，输出结果10


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g 
print(fancy_func(1, 2, 3, 4))
10


Sequential的混合式编程

In [3]:
import torch
from torch import nn 
from d2l import torch as d2l 

# 生产网络的工厂模式
def get_net():
    net = nn.Sequential(
            nn.Linear(512, 256), 
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[ 0.0861, -0.0049]], grad_fn=<AddmmBackward0>)

PyTorch基于命令式编程并且使用动态计算图，TorchScript允许用户使用纯命令式编程进行开发和调试，同时能够将大多数程序转换为符号式程序，以便在需要产品级计算性能和部署时使用

In [5]:
net = torch.jit.script(net)
net(x)

tensor([[ 0.0861, -0.0049]], grad_fn=<AddmmBackward0>)

通过混合式编程加速

In [6]:
# 上下文管理器，通过with语句使用
# 进入上下文: 执行__enter__方法，返回资源对象(如文件句柄)
# 退出上下文: 无论代码块是否抛出异常，都会执行__exit__方法，释放资源(如关闭文件)
class Benchmark:
    """用于度量运行时间"""
    def __init__(self, description='Done'):
        self.description = description
    
    def __enter__(self):
        self.timer = d2l.Timer()
        return self

    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')        

In [7]:
net = get_net()
with Benchmark('无torchscript'):
    for i in range(1000): net(x)

net = torch.jit.script(net)
with Benchmark('有torchscript'):
    for i in range(1000): net(x)

无torchscript: 0.3214 sec
有torchscript: 0.3109 sec


序列化

In [9]:
net.save('my_mlp')